In [1]:
account= "@UserName"
consumerKey = "........................."
consumerSecret = "........................."
accessToken = "........................."
accessTokenSecret = "........................."

In [2]:
# ライブラリのインポート
from textblob import TextBlob
import tweepy
import pandas as pd
import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from googletrans import Translator

# ツイートを監視し、自分へのメンションがあった場合、指定されたキーワードとツイート数からツイートを取得、分析し、結果をリプライするクラス
class SentimentAnalysisTweetBot(tweepy.Stream):
    
    def __init__(self, consumerKey, consumerSecret, accessToken, accessTokenSecret) : 
        super().__init__(consumerKey, consumerSecret, accessToken, accessTokenSecret)
        self.api = self.initTwitterAPI(consumerKey, consumerSecret, accessToken, accessTokenSecret)
        
    def on_status(self, status):
        print(status.user.screen_name + " : " +status.text)
        line = status.text.split(" ")
        keyword = line[1]
        noOfTweet = int(line[2])
        detail = False
        if len(line) > 3 :
            if line[3] == "detail" : detail = True
            else : detail = False
        tweets = self.getTweets(self.api, keyword, noOfTweet)
        tweet_df = self.sentimentAnalyzer(tweets)
        self.tweetResult(tweet_df, status.user.screen_name, status.id, detail)
    
        detail = False
        print("------fin------")
        print("searching...")
        
    def on_error(self, status_code):
        if status_code == 420:
            print("420 error")
            return False
        
    # APIの設定
    def initTwitterAPI(self, consumerKey, consumerSecret, accessToken, accessTokenSecret) :
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        return api
    
    def getTweets(self, api, keyword, noOfTweet):
        # キーワードの分割
        keyword_list = keyword.split(",")
        # リツイートは取得しない
        keyword += "-filter:retweets"
        # ツイートの取得
        tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)
        return tweets
    
        # sentimentのグループごとに数と割合を返す関数
    def count_values_in_column(self, data,feature):
        total=data.loc[:,feature].value_counts(dropna=False)
        percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
        return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

    
    def sentimentAnalyzer(self, tweets):
        # 元ツイート
        tweet_list_original = pd.DataFrame([tweet.text for tweet in tweets], columns=["original"])

        # 重複削除
        tweet_list_original.drop_duplicates()

        # 英語に翻訳
        translator = Translator()
        tweet_list_translated = [] # 英語に翻訳したツイート
        for tweet in tweet_list_original["original"] :
            if detect(tweet)=="en" :
                tweet_list_translated.append(tweet)
            else:
                tweet_list_translated.append(translator.translate(tweet, dest='en').text)

        tweet_list_translated = pd.DataFrame(tweet_list_translated)

        # ツイート情報
        tweet_df = pd.DataFrame()
        tweet_df["original"] = tweet_list_original
        tweet_df["translated"] = tweet_list_translated

        # 感情分析
        tweet_df[['polarity', 'subjectivity']] = tweet_df['translated'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        for index, row in tweet_df['translated'].iteritems():
            score = SentimentIntensityAnalyzer().polarity_scores(row)
            neg = score['neg']
            neu = score['neu']
            pos = score['pos']
            comp = score['compound']

            if neg > pos: 
                if neg>0.25 : tweet_df.loc[index, 'sentiment'] = "strong_negative"
                else : tweet_df.loc[index, 'sentiment'] = "negative"
            elif pos > neg: 
                if pos>0.25 : tweet_df.loc[index, "sentiment"] = "strong_positive"
                else : tweet_df.loc[index, "sentiment"] = "positive"
            else: tweet_df.loc[index, 'sentiment'] = "neutral"
            tweet_df.loc[index, 'neg'] = neg
            tweet_df.loc[index, 'neu'] = neu
            tweet_df.loc[index, 'pos'] = pos
            tweet_df.loc[index, 'compound'] = comp
        return tweet_df
    
#     分析結果をツイートする
    def tweetResult(self, tweet_df, screen_name, reply_id, detail):
        
        greeting = ""
        
        if detail==False :
            for index, row in tweet_df.iterrows():
                if row["sentiment"] == "strong_negative": tweet_df.loc[index, "sentiment"] = "アンチかも"
                else : tweet_df.loc[index, "sentiment"] = "アンチじゃないよ"
                    
            greeting += "アンチの割合を調べました\n"
        else :
            greeting += "詳しく調べました\n"
        
#         アンチコメントとみなされたツイートを表示する例
#         for index, row in tweet_df[tweet_df["sentiment"]=="アンチかも"].iterrows() : 
#             print(row.original)
#             print("-------------------------------------------")

        sentimentRate_df = self.count_values_in_column(tweet_df, "sentiment")
    
        print(sentimentRate_df)
    
        reply = "@"+str(screen_name)+"\n"+greeting+str(sentimentRate_df);
        self.api.update_status(reply, in_reply_to_status_id = reply_id)

In [3]:
# 使用方法
myStreamListener = SentimentAnalysisTweetBot(consumerKey, consumerSecret, accessToken, accessTokenSecret)

In [ ]:
myStreamListener.filter(track=[account])